In [348]:
import math
from torch.utils.tensorboard import SummaryWriter
from math import gamma
import os
import random
from tabnanny import verbose
import pandas as pd

import torch
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import StratifiedKFold

import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,ConcatDataset
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

## Datasets

In [361]:
train_tfm = transforms.Compose([
    # transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    ])

test_tfm = transforms.Compose([
    # transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    ])

In [350]:
dataset_train = datasets.MNIST(root='../data/', train=True,download=True,transform=train_tfm)

dataset_test = datasets.MNIST(root='./data/',train=False,download=True,transform=test_tfm)
                  

In [351]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

class MyDataset(Dataset):
    def __init__(self, feature, target=None, transform=None):
      self.X = feature
      self.Y = target
      self.transform = transform

    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      # img = Image.open(self.paths[index]).convert('RGB')
      if self.transform is not None:
          return self.transform(self.X[idx]), self.Y[idx]
      elif self.Y is None:
          return [self.X[idx]]
      return self.X[idx], self.Y[idx]



    # def __init__(self, X, Y=None,tfm=train_tfm):
    #     self.transform = tfm
    #     self.data = X
    #     if Y is not None:
    #         self.label = torch.LongTensor(Y)
    #     else:
    #         self.label = None

    # def __getitem__(self, idx):
    #     if self.label is not None:
    #         return self.data[idx], self.label[idx]
    #     else:
    #         return self.data[idx]

    # def __len__(self):
    #     return len(self.data)






## Model Structure

In [352]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        # self.resnet50 = models.resnet50(pretrained=False,num_classes=10)
        self.resnet50 = models.resnet50()

        # self.fc = nn.Sequential(
        #     nn.Linear(16,64),
        #     nn.BatchNorm1d(64),
        #     nn.ReLU(),
        #     nn.Dropout(0.25),
        #     nn.Linear(64,128),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #     nn.Dropout(0.25),
        #     nn.Linear(128,10),
        # )

    def forward(self, x):
        out = self.resnet50(x)
        return out

## Hyper-parameter


In [353]:
# fix random seed
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'epochs': 3000,     # Number of epochs.  
    'folds' : 10,          
    'batch_size': 32, 
    'weight_decay': 1e-4,
    'learning_rate': 1e-4,              
    'early_stop': 5,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss(reduction='mean')
# model = MNISTNet().to(device)
optimizer = optim.AdamW(model.parameters(),lr=config['learning_rate'],weight_decay=config['weight_decay'])
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=2, T_mult=2, eta_min=config['learning_rate'])




In [354]:
# def reset_weights(m):
#     if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#         m.reset_parameters()

In [355]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## Dataloader

In [356]:
# Set seed for reproducibility
def same_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
same_seed(config['seed'])
# channel = 3     # 1 3
# img_size = 32   # 36 32
# train_data = FooDataset(num_data=32, shape=(channel, img_size, img_size))
dataset_train = datasets.MNIST(root='../data/', train=True,download=True,transform=train_tfm)
# print(len(dataset_train))
dataset_test = datasets.MNIST(root='./data/',train=False,download=True,transform=test_tfm)

## validation set
valid_ratio = 0.2
valid_set_size = int(valid_ratio * len(dataset_train)) 
train_set_size = len(dataset_train) - valid_set_size
dataset_train, dataset_valid = torch.utils.data.random_split(dataset_train, [train_set_size, valid_set_size])

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(dataset_train, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(dataset_valid, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset_test, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

## Train/Test functions

In [357]:
best_acc = 0.0
# train(train_loader, valid_loader, model, config, device)
def train(train_loader, valid_loader, model, config, device):
    writer = SummaryWriter() # Writer of tensoboard.
    n_epochs, best_loss, step, early_stop_count = config['epochs'], math.inf, 0, 0

    
    for epoch in range(config['epochs']):
    #-----------------------train----------------------#
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for data, label in train_pbar: ##each batch 
            optimizer.zero_grad()               # Set gradient to zero.
            data, label = data.to(device), label.to(device)   # Move your data to device. 
            
            target = model(data)             
            loss = criterion(target, label)
            loss.backward()                     # Compute gradient(backpropagation).

            optimizer.step()                    # Update parameters.
            scheduler.step()

            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

    #-----------------------validation----------------------#

        model.eval() # Set your model to evaluation mode.
        loss_record = []

        for data, label in valid_loader:
            data, label = data.to(device), label.to(device)   # Move your data to device. 
            with torch.no_grad():
              target = model(data)             
              loss = criterion(target, label)
            loss_record.append(loss.detach().item())
        mean_valid_loss = sum(loss_record)/len(loss_record)


    #-----------------------print result----------------------#
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            train_loss_now = mean_train_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            print('best loss of validation is',best_loss,',at epoch',epoch)
            print('loss of training set associated is',train_loss_now)
            return


In [358]:
def predict(test_loader, model, device): #final test time: without label
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds


## Start training

In [359]:
model = MNISTNet().to(device)

# print(model)

In [360]:
train(train_loader, valid_loader, model, config, device)

  0%|          | 0/1500 [00:00<?, ?it/s]


RuntimeError: ignored

## Plot tensorboard

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

## Make prediction

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    preds = predict(test_loader, model, device)
    for i, y in enumerate(preds):
        f.write('{},{}\n'.format(i, y))